# `pipestat` CLI

Before following this tutorial please make sure you're familiar with "`pipestat` Python API" tutorial.

## Usage reference

To learn about the usage `pipestat` usage use `--help`/`-h` option on any level:

In [1]:
pipestat -h

version: 0.0.1
usage: pipestat [-h] [--version] [--silent] [--verbosity V] [--logdev]
                {report,inspect,remove} ...

pipestat - pipeline results reported

positional arguments:
  {report,inspect,remove}
    report              Report a result.
    inspect             Inspect a database.
    remove              Remove a result.

optional arguments:
  -h, --help            show this help message and exit
  --version             show program's version number and exit
  --silent              Silence logging. Overrides verbosity.
  --verbosity V         Set logging level (1-5 or logging module level name)
  --logdev              Expand content of logging message format.

...


In [2]:
pipestat report -h

usage: pipestat report [-h] -d DB -n N -i ID -v V -t T [-o] [-s]

Report a result.

optional arguments:
  -h, --help            show this help message and exit
  -d DB, --database DB  database to store results in
  -n N, --name N        name of the pipeline to report result for
  -i ID, --id ID        id of the result to report
  -v V, --value V       value of the result to report
  -t T, --type T        type of the result to report
  -o, --overwrite       whether the result should override existing ones in
                        case of name clashes
  -s, --strict-type     whether the result should be casted to the class
                        required by the declared type and the effect of this
                        operation verified


In [3]:
pipestat remove -h

usage: pipestat remove [-h] -d DB -n N -i ID

Remove a result.

optional arguments:
  -h, --help            show this help message and exit
  -d DB, --database DB  database to store results in
  -n N, --name N        name of the pipeline to report result for
  -i ID, --id ID        id of the result to report


In [4]:
pipestat inspect -h

usage: pipestat inspect [-h] -d DB -n N

Inspect a database.

optional arguments:
  -h, --help            show this help message and exit
  -d DB, --database DB  database to store results in
  -n N, --name N        name of the pipeline to report result for


## Usage demonstration

### Reporting

Naturally, the command line interface provides access to all the Python API functionalities of `pipestat`. So, for example, to report a result and back the object by a file use:

In [5]:
temp_file=`mktemp`
pipestat report -d $temp_file -n test -i files_count -t integer -v 100

Cached new 'test' record: files_count=100(integer)
Wrote 1 cached records: {'test': {'files_count': {'value': '100', 'type': 'integer'}}}


The result has been reported and the database file has been updated:

In [6]:
cat $temp_file

test:
  files_count:
    value: '100'
    type: integer


Let's report another result:

In [7]:
pipestat report -d $temp_file -n test -i files_size -t float -v 11.1

Cached new 'test' record: files_size=11.1(float)
Wrote 1 cached records: {'test': {'files_size': {'value': '11.1', 'type': 'float'}}}


In [8]:
cat $temp_file

test:
  files_count:
    value: '100'
    type: integer
  files_size:
    value: '11.1'
    type: float


### Inspection

In [9]:
pipestat inspect -d $temp_file -n test

PipeStatManager (test)
Database length: 2
Database: {'test': {'files_count': {'value': '100', 'type': 'integer'}, 'files_size': {'value': '11.1', 'type': 'float'}}}
Cache length: 0
Cache: {}


### Removal

In [10]:
pipestat remove -d $temp_file -n test -i files_size

In [11]:
cat $temp_file

test:
  files_count:
    value: '100'
    type: integer


In [12]:
pipestat inspect -d $temp_file -n test

PipeStatManager (test)
Database length: 1
Database: {'test': {'files_count': {'value': '100', 'type': 'integer'}}}
Cache length: 0
Cache: {}
